In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pickle

In [2]:
# read and preprocess dataset
def data_prep(file):
    # Load January data
    df = pd.read_parquet(file)
    # get the total time of the trip  by taking the time difference
    df["duration"] = df["dropOff_datetime"] - df["pickup_datetime"]
    # convert the time duration from days to minutes format
    df["duration"] = df["duration"].apply(lambda dur:dur.total_seconds()/60)
    # filter the dataset based on duration (in mins)
    df = df[(df["duration"]>1)&(df["duration"]<=60)]
    # replace NaN value in some columns with value -1  
    df[["PUlocationID", "DOlocationID"]] = df[["PUlocationID", "DOlocationID"]].fillna(-1)
    # select the pick up location, drop off location and ride duration columns
    df_preped = df[["PUlocationID", "DOlocationID", "duration"]]
    
    return df_preped

In [3]:
train_df = data_prep('/home/nwoke/Documents/git_cloned/Github/mlops_101/data/fhv_tripdata_2021-01.parquet')
train_df.shape

(1109205, 3)

In [9]:
test_df = data_prep('/home/nwoke/Documents/git_cloned/Github/mlops_101/data/fhv_tripdata_2021-02.parquet')
test_df.shape

(989674, 3)

In [5]:
categorical = ["PUlocationID", "DOlocationID"]

In [6]:
# one hot encode the features by using the dict vectorizer
dv = DictVectorizer()

train_df[categorical] = train_df[categorical].astype(str)
train_df_ = train_df[categorical].to_dict(orient="records")
X_train = dv.fit_transform(train_df_)

test_df[categorical] = test_df[categorical].astype(str)
test_df_ = test_df[categorical].to_dict(orient="records")
X_val = dv.transform(test_df_)


In [7]:
# get the labels
y_train = train_df["duration"].values
y_val = test_df["duration"].values

In [8]:
# model fitting
lr = LinearRegression()
lr.fit(X_train, y_train)

# predict the trip duration for feburary data
y_pred = lr.predict(X_val)

# model evaluation
mean_squared_error(y_pred, y_val, squared=False)

11.008906627791399

In [12]:
# dump the model and one-hot encoding object as a pickle file
with open("/home/nwoke/Documents/git_cloned/Github/mlops_101/week1/models/linear_model.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)